In [1]:
import pandas as pd

In [2]:
file = "HeroesOfPymoli/purchase_data.json"
rawdata_df = pd.read_json(file)
rawdata_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [3]:
PlayerCount = len(rawdata_df["SN"].unique())
playercount_disp = pd.DataFrame({"Player Count": [PlayerCount]})
playercount_disp

,Player Count
0,573


In [4]:
# Data manipulaton
UniqueItems = len(rawdata_df["Item Name"].unique())
AvgPurchase = rawdata_df["Price"].mean()
NumPurchase = len(rawdata_df["Item Name"])
Revenue = rawdata_df["Price"].sum()

# Create new DataFrame
Purchasing_Analysis_Total = pd.DataFrame({"Number of Unique Items": [UniqueItems],
                                           "Average Price": [AvgPurchase],
                                           "Number of Purchases": [NumPurchase],
                                           "Total Revenue": [Revenue]})

# DataFrame formatting
Purchasing_Analysis_Total["Average Price"] = Purchasing_Analysis_Total["Average Price"].map("${:.2f}".format)
Purchasing_Analysis_Total["Total Revenue"] = Purchasing_Analysis_Total["Total Revenue"].map("${:.2f}".format)
Purchasing_Analysis_Total = Purchasing_Analysis_Total[["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"]]

Purchasing_Analysis_Total

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$2.93,780,$2286.33


In [5]:
# Data manipulation
duplicate = rawdata_df.drop_duplicates(subset='SN', keep="first")
TotalGen = duplicate["Gender"].count()
MaleGen = duplicate["Gender"].value_counts()['Male']
FemaleGen = duplicate["Gender"].value_counts()['Female']
NonGen = TotalGen - MaleGen - FemaleGen

MalePer = (MaleGen / TotalGen) * 100
FemalePer = (FemaleGen / TotalGen) * 100
NonPer = (NonGen / TotalGen) * 100

# Create new DataFrame
Gender_Demo = pd.DataFrame({"": ['Male', 'Female', 'Other/Non-Disclosed'],
                            "Percentage of Players": [MalePer, FemalePer, NonPer],
                            "Total Count": [MaleGen, FemaleGen, NonGen]})

# DataFrame formatting
Gender_Demo["Percentage of Players"] = Gender_Demo["Percentage of Players"].map("{:.2f}%".format)
Gender_Demo = Gender_Demo.set_index('')
Gender_Demo

,Percentage of Players,Total Count
,,
Male,81.15%,465
Female,17.45%,100
Other/Non-Disclosed,1.40%,8


In [6]:
# Group by Gender
grouped_df = rawdata_df.groupby(["Gender"])

# Data Manipulation
purchCount = grouped_df["SN"].count()
purchPrice = grouped_df["Price"].mean()
purchValue = grouped_df["Price"].sum()

# Normalize data by deleting all duplicates adn resort
duplicate = rawdata_df.drop_duplicates(subset='SN', keep="first")
grouped_dup = duplicate.groupby(["Gender"])

# Find normalized data
purchNorm = (grouped_df["Price"].sum() / grouped_dup["SN"].count())

# Create new DataFrame
Purch_Anal_Gen = pd.DataFrame({"Purchase Count": purchCount,
                              "Average Purchase Price": purchPrice,
                              "Total Purchase Value": purchValue,
                              "Normalized Totals": purchNorm})

# DataFrame formatting
Purch_Anal_Gen["Average Purchase Price"] = Purch_Anal_Gen["Average Purchase Price"].map("${:.2f}".format)
Purch_Anal_Gen["Total Purchase Value"] = Purch_Anal_Gen["Total Purchase Value"].map("${:.2f}".format)
Purch_Anal_Gen["Normalized Totals"] = Purch_Anal_Gen["Normalized Totals"].map("${:.2f}".format)
Purch_Anal_Gen = Purch_Anal_Gen[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
Purch_Anal_Gen

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,$1867.68,$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


In [7]:
# Binning
bins = [0,10,15,20,25,30,35,40,200]
binLab = ['Under 10', '10 - 14', '15 - 19', '20 - 24', '25 - 29', '30 - 34', '35 - 39', 'Over 40']

# Add bins to new dataframe and groupby
binner_df = rawdata_df.copy()
binner_df["Age Groups"] = pd.cut(binner_df["Age"], bins, labels=binLab)
group_bin = binner_df.groupby(["Age Groups"])

# Data manipulation
binnerCount = group_bin["SN"].count()
countTotal = rawdata_df["SN"].count()
percentage = (binnerCount / countTotal) * 100
percentage

# Create new DataFrame
Age_Perc = pd.DataFrame({"Total Count": binnerCount,
                         "Percentage of Players": percentage})

# DataFrame formatting
Age_Perc["Percentage of Players"] = Age_Perc["Percentage of Players"].map("{:.2f}%".format)
Age_Perc.head(10)

,Percentage of Players,Total Count
Age Groups,,
Under 10,4.10%,32
10 - 14,10.00%,78
15 - 19,23.59%,184
20 - 24,39.10%,305
25 - 29,9.74%,76
30 - 34,7.44%,58
35 - 39,5.64%,44
Over 40,0.38%,3


In [8]:
# Binning
bins = [0,10,15,20,25,30,35,40,200]
binLab = ['Under 10', '10 - 14', '15 - 19', '20 - 24', '25 - 29', '30 - 34', '35 - 39', 'Over 40']

# Add bins to new dataframe and groupby
binning_df = rawdata_df.copy()
binning_df["Age Groups"] = pd.cut(binning_df["Age"], bins, labels=binLab)
binColumn = pd.cut(binning_df["Age"], bins, labels=binLab)
grouped_bin = binning_df.groupby(["Age Groups"])

# Data Manipulation
binPCount = grouped_bin["Age"].count()
binPAver = grouped_bin["Price"].mean()
binPTotal = grouped_bin["Price"].sum()

# Normalize data by deleting duplicates for new counts
binningduplicate = rawdata_df.drop_duplicates(subset='SN', keep="first")
binningduplicate["Age Groups"] = pd.cut(binningduplicate["Age"], bins, labels=binLab)
binningduplicate = binningduplicate.groupby(["Age Groups"])

binningNorm = (grouped_bin["Price"].sum() / binningduplicate["SN"].count())
binningNorm

# Create new DF and format
Age_Demo = pd.DataFrame({"Purchase Count": binPCount,
                         "Average Purchase Price": binPAver,
                         "Total Purchase Value": binPTotal,
                         "Normalized Totals": binningNorm})

Age_Demo["Average Purchase Price"] = Age_Demo["Average Purchase Price"].map("${:.2f}".format)
Age_Demo["Total Purchase Value"] = Age_Demo["Total Purchase Value"].map("${:.2f}".format)
Age_Demo["Normalized Totals"] = Age_Demo["Normalized Totals"].map("${:.2f}".format)
Age_Demo = Age_Demo[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
Age_Demo.head(10)

C:\Users\domor\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Groups,,,,
Under 10,32,$3.02,$96.62,$4.39
10 - 14,78,$2.87,$224.15,$4.15
15 - 19,184,$2.87,$528.74,$3.80
20 - 24,305,$2.96,$902.61,$3.86
25 - 29,76,$2.89,$219.82,$4.23
30 - 34,58,$3.07,$178.26,$4.05
35 - 39,44,$2.90,$127.49,$5.10
Over 40,3,$2.88,$8.64,$2.88


In [9]:
# Data Manipulation
groupedBySN = rawdata_df.groupby(["SN"])
groupedSNCount = groupedBySN["Item ID"].count()
groupedSNTotal = groupedBySN["Price"].sum()
groupedSNAvg = (groupedSNTotal / groupedSNCount)

# Build DF and format
SN_Demo = pd.DataFrame({"Purchase Count": groupedSNCount,
                         "Average Purchase Price": groupedSNAvg,
                         "Total Purchase Value": groupedSNTotal})

SN_Demo = SN_Demo.sort_values("Total Purchase Value", ascending=False) 
SN_Demo["Average Purchase Price"] = SN_Demo["Average Purchase Price"].map("${:.2f}".format)
SN_Demo["Total Purchase Value"] = SN_Demo["Total Purchase Value"].map("${:.2f}".format)
SN_Demo = SN_Demo[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
SN_Demo.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


In [10]:
# Data manipulation
groupItem = rawdata_df.groupby(["Item ID", "Item Name"])
groupItemC = groupItem["SN"].count()
groupPriceSum = groupItem["Price"].sum()
groupItemP = (groupPriceSum / groupItemC)
groupItemV = (groupItemP * groupItemC)

# New DF with formatting
Pop_Item = pd.DataFrame({"Purchase Count": groupItemC,
                          "Item Price": groupItemP,
                          "Total Purchase Value": groupItemV})

Pop_Item = Pop_Item.sort_values("Purchase Count", ascending=False) 
Pop_Item["Item Price"] = Pop_Item["Item Price"].map("${:.2f}".format)
Pop_Item["Total Purchase Value"] = Pop_Item["Total Purchase Value"].map("${:.2f}".format)
Pop_Item = Pop_Item[["Purchase Count", "Item Price", "Total Purchase Value"]]
Pop_Item.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


In [11]:
# Data manipulation
groupedItem = rawdata_df.groupby(["Item ID", "Item Name"])
groupedItemC = groupedItem["Gender"].count()
groupedSum = groupedItem["Price"].sum()
groupedItemP = (groupedSum / groupedItemC)

# Make a new DF and format
Pop_Val = pd.DataFrame({"Purchase Count": groupedItemC,
                          "Item Price": groupedItemP,
                          "Total Purchase Value": groupedSum})

Pop_Val = Pop_Val.sort_values("Total Purchase Value", ascending=False) 
Pop_Val["Item Price"] = Pop_Val["Item Price"].map("${:.2f}".format)
Pop_Val["Total Purchase Value"] = Pop_Val["Total Purchase Value"].map("${:.2f}".format)
Pop_Val = Pop_Val[["Purchase Count", "Item Price", "Total Purchase Value"]]
Pop_Val.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
